In [2]:
import sys
sys.path

['/usr/src/app/notebooks',
 '/usr/src/app',
 '/usr/src/app/gee_lib',
 '/usr/src/app/ex_ante_lib',
 '/usr/src/app/forestry_carbon_arr',
 '/usr/src/app/fastapi-gee-service',
 '/usr/local/lib/python310.zip',
 '/usr/local/lib/python3.10',
 '/usr/local/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/site-packages']

In [3]:
import ee

In [4]:
from gee_lib.osi.auth import initialize_gee, check_gee_initialization
# Initialize GEE
authenticate = initialize_gee(project_id='ee-iwansetiawan')

# Verify it's working
if check_gee_initialization():
    print("Ready to use GEE!")

✓ GEE Initialized successfully
  Service Account: earth-engine-land-eligibility@ee-iwansetiawan.iam.gserviceaccount.com
  Project ID: ee-iwansetiawan
  Credentials Path: /usr/src/app/user_id.json


In [5]:
import json
import os

### need improvement on how to manage this config, we also in the config the path to the static file
# maybe we need to not put the path in the config.json, don't know yet
# Move to the parent directory of the current script
parent_dir = os.getcwd()

# Construct the absolute path to the JSON file in the 'input' folder
json_path= os.path.join(parent_dir, '00_input', 'balaban_conf.json')

# Read and load the JSON data from the file
with open(json_path, 'r') as file:
    config = json.load(file)

print('config---> ',config)

config--->  {'I_satellite': 'Sentinel', 'pca_scaling': 1, 'tileScale': 2, 'AOI_path': './00_input/LPHD Belaban Rayak.shp', 'OID_field_name': 'id', 'input_training': './00_input/traning_point_merged_belaban.shp', 'algo_ml_selected': 'gbm', 'date_start_end': ['2024-1-1', '2024-5-31'], 'project_name': 'belaban_rayak', 'super_pixel_size': 3, 'region': 'asia', 'pixel_number': 3, 'year_start_loss': 14, 'tree_cover_forest': 30, 'band_name_image': 'Class', 'cloud_cover_threshold': 60, 'crs_input': 'EPSG:4326', 'IsThermal': False, 'fcd_selected': 21, 'high_forest': 65, 'yrf_forest': 45, 'shrub_grass': 45, 'open_land': 30, 'ndwi_hi_sentinel': 0.05, 'ndwi_hi_landsat': 0.1, 'ndwi_hi_planet': -0.2}


In [6]:
## import modules
import osi
import pandas as pd
import geemap

from osi.utils.main import validate_aoi
# convert the modules for image collection (cloudless masking, compositing, reducer etc)
from osi.image_collection.main import ImageCollection
from osi.spectral_indices.spectral_analysis import SpectralAnalysis
from osi.spectral_indices.utils import normalization_100
from osi.hansen.historical_loss import HansenHistorical
from osi.classifying.assign_zone import AssignClassZone
from osi.legends.utils import convert_to_legend_items
from osi.legends.main import LegendsBuilder
from osi.obia.main import OBIASegmentation
from osi.ml.main import LandcoverML
from osi.fcd.main_fcd import FCDCalc
from osi.pca.pca_gee import PCA
from osi.hansen.historical_loss import HansenHistorical
from osi.classifying.assign_zone import AssignClassZone
from osi.arcpy.utils import safe_get_data_source


AOIt_shp_plot = geemap.shp_to_ee(config['AOI_path'])
crs_input = config['crs_input']
I_satellite = config['I_satellite']
project_name = config['project_name']

start_date = config['date_start_end'][0]
end_date = config['date_start_end'][1]

layer_name_image_mosaick = f'image_mosaick_result_ee_{project_name}'

AOI = AOIt_shp_plot
config['AOI'] = AOI

ndwi_hi = 0.1
if config['I_satellite'] == 'Landsat':
    ndwi_hi = config['ndwi_hi_landsat']
elif I_satellite == 'Sentinel':
    ndwi_hi = config['ndwi_hi_sentinel']
elif I_satellite == 'Planet':
    ndwi_hi = config['ndwi_hi_planet']

### Masking and overlay and area helper Make an image out of the AOI area attribute -> convert featurecollection into raster (image) for overlaying tools
OID = config['OID_field_name']
AOI_img = AOI.filter(ee.Filter.notNull([OID])).reduceToImage(
    properties= [OID],
    reducer= ee.Reducer.first()
)

In [7]:
# now starting to do analysis
# initiate instance class for the image collection and later mosaicking
classInputCollection = ImageCollection(I_satellite=I_satellite,
                                       AOI=AOI, 
                                       date_start_end=config['date_start_end'], 
                                       cloud_cover_threshold = config['cloud_cover_threshold'],
                                       region=config['region'])

In [8]:
# run the method from image collection loaded, cloudless compositing until to image_mosaick
image_mosaick = classInputCollection.image_mosaick()

selecting Sentinel images


In [9]:
# Generate Map IDs
### Defining params visualization and layer name
if config['I_satellite'] == 'Planet':
    # true color {"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}
    # nir veg color {"bands":["red","nir","blue"],"min":0,"max":0.6,"gamma":1.5 }
    image_mosaick, vis_param, layer_name = image_mosaick,{f'{I_satellite} mosaicked - {start_date}-{end_date} VegColor':{"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}}, f'{I_satellite} mosaicked - {start_date}-{end_date} VegColor'
else:
    image_mosaick, vis_param, layer_name = image_mosaick,{f'{I_satellite} mosaicked - {start_date}-{end_date} VegColor':{'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 0.6, 'gamma': 1.5 }}, f'{I_satellite} mosaicked - {start_date}-{end_date} VegColor'
   

print("Generating GEE Map IDs...")

map_ids = {}
map_ids[layer_name] = image_mosaick.getMapId(vis_param[layer_name])

vis_params = {}
vis_params[layer_name] = vis_param[layer_name]

print("✓ Map IDs generated for all layers")

# Display tile URLs
for layer_name, map_id_dict in map_ids.items():
    tile_url = map_id_dict['tile_fetcher'].url_format
    print(f"\n{layer_name.upper()}:")
    print(f"  Tile URL: {tile_url}")



Generating GEE Map IDs...
✓ Map IDs generated for all layers

SENTINEL MOSAICKED - 2024-1-1-2024-5-31 VEGCOLOR:
  Tile URL: https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/c105e89d63ba8761ee46d0d2b2620ffa-5810a68fafc56ad95c0d8196d869c4f1/tiles/{z}/{x}/{y}


In [10]:
map_ids

{'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': {'mapid': 'projects/ee-iwansetiawan/maps/c105e89d63ba8761ee46d0d2b2620ffa-5810a68fafc56ad95c0d8196d869c4f1',
  'token': '',
  'tile_fetcher': <ee.data.TileFetcher at 0x7f3fa3f95570>,
  'image': <ee.image.Image at 0x7f3f74315f00>}}

In [11]:
## META data layers

layers_data = {
    'project_name': project_name,
}

# # Option 1: Simple tile URLs only
# map_layers = {
#     layer_name: map_ids[layer_name]['tile_fetcher'].url_format}

map_layers = {}

layers_data = {'project_name': 'Sentinel-2 Cloudless Composite Analysis'}

# Add each layer from your GEE analysis
for layer_name, map_id_dict in map_ids.items():
    map_layers[layer_name] = {
        'tile_url': map_id_dict['tile_fetcher'].url_format,
        'name': layer_name.replace('_', ' ').title(),
        'description': f'{layer_name.upper()} visualization from GEE analysis',
        'vis_params': vis_params[layer_name]
    }

In [12]:
map_layers

{'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': {'tile_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/c105e89d63ba8761ee46d0d2b2620ffa-5810a68fafc56ad95c0d8196d869c4f1/tiles/{z}/{x}/{y}',
  'name': 'Sentinel Mosaicked - 2024-1-1-2024-5-31 Vegcolor',
  'description': 'SENTINEL MOSAICKED - 2024-1-1-2024-5-31 VEGCOLOR visualization from GEE analysis',
  'vis_params': {'bands': ['swir2', 'nir', 'red'],
   'min': 0,
   'max': 0.6,
   'gamma': 1.5}}}

In [ ]:
# import sys
# sys.path.append('/usr/src/app/fastapi-gee-service')
from gee_integration import process_gee_to_mapstore

# Import the AOI fix function properly
from gee_lib.osi.utils import process_aoi_geometry

# Fix 1: Use correct map layers
print(f"\n🔍 Available map layers:")
print(f"   map_layers: {len(map_layers)}")

# Fix 2: Fix AOI coordinates using the existing function
print(f"\n🔧 Fixing AOI coordinates...")
AOI_geom = AOI.geometry()
aoi_info = process_aoi_geometry(AOI_geom)

print(f"✅ Fixed AOI Info:")
print(f"   Bbox: {aoi_info['bbox']}")
print(f"   Center: {aoi_info['center']}")

print("🚀 Processing GEE Analysis to MapStore...")
print(f"   Project: {layers_data['project_name']}")
print(f"   Layers: {len(map_layers)}")
print(f"   AOI: {aoi_info['bbox']}")

# Process the complete integration
result = process_gee_to_mapstore(
    map_layers=map_layers,
    project_name=layers_data['project_name'],
    aoi_info=aoi_info,
    clear_cache_first=False,
    fastapi_url="http://fastapi:8000"  # Internal Docker network , this should be in localhost, but we can override as well
)

In [10]:
# !cat /usr/src/app/mapstore/config/localConfig.json

In [14]:
# =============================================================================
# Convert Your Existing map_layers to CSW Records
# =============================================================================

print("🔄 Converting Your map_layers to CSW Records")
print("=" * 60)

def map_layers_to_csw_records(map_layers, aoi_info=None, fastapi_url="http://localhost:8001"):
    """
    Convert your existing map_layers to CSW records
    
    Args:
        map_layers: Your existing map_layers dictionary
        aoi_info: AOI information with bbox
        fastapi_url: FastAPI URL for TMS endpoints
        
    Returns:
        List of CSW records
    """
    csw_records = []
    
    for layer_name, layer_info in map_layers.items():
        # Clean layer name for URL
        import re
        clean_name = re.sub(r'[^a-zA-Z0-9_]', '_', layer_name)
        clean_name = re.sub(r'_+', '_', clean_name).strip('_')
        
        # Create TMS URL
        tms_url = f"{fastapi_url}/tms/dynamic/{clean_name}/{{z}}/{{x}}/{{y}}.png"
        
        # Extract information
        title = layer_info.get('name', layer_name)
        description = layer_info.get('description', f'Layer: {layer_name}')
        tile_url = layer_info.get('tile_url', '')
        vis_params = layer_info.get('vis_params', {})
        
        # Create CSW record
        record = {
            "dc:title": title,
            "dc:description": description,
            "dc:type": "service",
            "dc:format": "image/png",
            "ows:ServiceType": "TMS",
            "ows:ServiceTypeVersion": "1.0.0",
            "tms:URLTemplate": tms_url,
            "tms:MinZoom": 0,
            "tms:MaxZoom": 18,
            "tms:CRS": "EPSG:3857",
            "gee:LayerName": layer_name,
            "gee:OriginalTileURL": tile_url,
            "gee:VisParams": vis_params,
            "gee:Source": "map_layers"
        }
        
        # Add bounding box if available
        if aoi_info and aoi_info.get('bbox'):
            bbox = aoi_info['bbox']
            record["ows:BoundingBox"] = {
                "ows:CRS": "EPSG:4326",
                "ows:LowerCorner": f"{bbox['minx']} {bbox['miny']}",
                "ows:UpperCorner": f"{bbox['maxx']} {bbox['maxy']}"
            }
        
        csw_records.append(record)
    
    return csw_records

# Convert your existing map_layers to CSW records
print("\n1️⃣ Converting your map_layers to CSW records...")
try:
    csw_records = map_layers_to_csw_records(
        map_layers=map_layers,
        aoi_info=aoi_info if 'aoi_info' in locals() else None,
        fastapi_url="http://fastapi:8000"
    )
    
    print(f"   ✅ Converted {len(csw_records)} layers to CSW records")
    
    # Show the CSW records
    for i, record in enumerate(csw_records):
        print(f"\n   📋 CSW Record {i+1}:")
        print(f"      Title: {record['dc:title']}")
        print(f"      Description: {record['dc:description']}")
        print(f"      TMS URL: {record['tms:URLTemplate']}")
        print(f"      Original URL: {record['gee:OriginalTileURL']}")
        if 'ows:BoundingBox' in record:
            bbox = record['ows:BoundingBox']
            print(f"      BBox: {bbox['ows:LowerCorner']} to {bbox['ows:UpperCorner']}")
        
except Exception as e:
    print(f"   ❌ Error: {e}")

# Example 2: Create a CSW service response
print("\n2️⃣ Creating CSW service response...")
try:
    # Create a CSW GetRecordsResponse format
    csw_response = {
        "csw:GetRecordsResponse": {
            "csw:SearchResults": {
                "numberOfRecordsMatched": len(csw_records),
                "numberOfRecordsReturned": len(csw_records),
                "nextRecord": 0,
                "csw:Record": csw_records
            }
        }
    }
    
    print(f"   ✅ Created CSW response with {len(csw_records)} records")
    print(f"   Total matched: {csw_response['csw:GetRecordsResponse']['csw:SearchResults']['numberOfRecordsMatched']}")
    
except Exception as e:
    print(f"   ❌ Error: {e}")

# Example 3: Use CSW records to enhance your workflow
print("\n3️⃣ Using CSW records to enhance your workflow...")
try:
    # Create enhanced map_layers with CSW metadata
    enhanced_map_layers = {}
    
    for record in csw_records:
        layer_name = record['gee:LayerName']
        original_info = map_layers[layer_name]
        
        # Enhance with CSW metadata
        enhanced_map_layers[layer_name] = {
            **original_info,  # Keep original info
            'csw_record': record,
            'tms_url': record['tms:URLTemplate'],
            'csw_title': record['dc:title'],
            'csw_description': record['dc:description'],
            'bbox': record.get('ows:BoundingBox', {}),
            'source': 'csw_enhanced'
        }
    
    print(f"   ✅ Enhanced {len(enhanced_map_layers)} layers with CSW metadata")
    
    # Show enhanced layers
    for layer_name, layer_info in enhanced_map_layers.items():
        print(f"\n   🔗 Enhanced Layer: {layer_name}")
        print(f"      Original: {layer_info.get('tile_url', 'N/A')}")
        print(f"      TMS: {layer_info.get('tms_url', 'N/A')}")
        print(f"      CSW Title: {layer_info.get('csw_title', 'N/A')}")
        print(f"      Source: {layer_info.get('source', 'N/A')}")
        
except Exception as e:
    print(f"   ❌ Error: {e}")

# Example 4: Add CSW records as TMS layers to MapStore
print("\n4️⃣ Adding CSW records as TMS layers to MapStore...")
try:
    from gee_integration import add_tms_layer_to_mapstore
    
    added_layers = []
    failed_layers = []
    
    for record in csw_records:
        layer_name = record['gee:LayerName']
        title = record['dc:title']
        tms_url = record['tms:URLTemplate']
        
        try:
            # Add as TMS layer to MapStore
            result = add_tms_layer_to_mapstore(
                layer_name=layer_name,
                layer_url=tms_url,
                layer_title=title,
                use_fastapi_proxy=True,
                fastapi_pub_url="http://localhost:8001"
            )
            
            if result['status'] == 'success':
                added_layers.append({
                    'layer_name': layer_name,
                    'title': title,
                    'service_id': result['service_id']
                })
            else:
                failed_layers.append({
                    'layer_name': layer_name,
                    'error': result.get('message', 'Unknown error')
                })
                
        except Exception as e:
            failed_layers.append({
                'layer_name': layer_name,
                'error': str(e)
            })
    
    print(f"   ✅ Added {len(added_layers)} layers to MapStore")
    print(f"   ❌ Failed {len(failed_layers)} layers")
    
    if added_layers:
        print("   Added layers:")
        for layer in added_layers:
            print(f"     - {layer['title']} (ID: {layer['service_id']})")
    
    if failed_layers:
        print("   Failed layers:")
        for layer in failed_layers:
            print(f"     - {layer['layer_name']}: {layer['error']}")
            
except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n🎯 map_layers to CSW Summary:")
print("   ✅ Convert existing map_layers to CSW records")
print("   ✅ Add CSW metadata to your layers")
print("   ✅ Create TMS layers from CSW records")
print("   ✅ Integrate with MapStore")
print("   ✅ Use your existing GEE analysis results")
print("   ✅ Standard CSW format for interoperability")


INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Adding TMS layer: Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor
INFO:gee_integration:✅ TMS layer added successfully
INFO:gee_integration:   Service ID: gee_tms_Sentinel_mosaicked_2024_1_1_2024_5_31_VegColor
INFO:gee_integration:   Layer: Sentinel Mosaicked - 2024-1-1-2024-5-31 Vegcolor
INFO:gee_integration:   URL: http://localhost:8001/tms/dynamic/Sentinel_mosaicked_2024_1_1_2024_5_31_VegColor/{z}/{x}/{y}.png


🔄 Converting Your map_layers to CSW Records

1️⃣ Converting your map_layers to CSW records...
   ✅ Converted 1 layers to CSW records

   📋 CSW Record 1:
      Title: Sentinel Mosaicked - 2024-1-1-2024-5-31 Vegcolor
      Description: SENTINEL MOSAICKED - 2024-1-1-2024-5-31 VEGCOLOR visualization from GEE analysis
      TMS URL: http://fastapi:8000/tms/dynamic/Sentinel_mosaicked_2024_1_1_2024_5_31_VegColor/{z}/{x}/{y}.png
      Original URL: https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/c105e89d63ba8761ee46d0d2b2620ffa-964e7923f6e012b42d9c4ed31dc19a53/tiles/{z}/{x}/{y}

2️⃣ Creating CSW service response...
   ✅ Created CSW response with 1 records
   Total matched: 1

3️⃣ Using CSW records to enhance your workflow...
   ✅ Enhanced 1 layers with CSW metadata

   🔗 Enhanced Layer: Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor
      Original: https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/c105e89d63ba8761ee46d0d2b2620ffa-964e7923f6e012b42d9c4ed3

In [ ]:
# !cat /usr/src/app/mapstore/configs/localConfig.json

In [20]:
# AOI.getInfo()

In [ ]:
## EXAMPLE TO PUSH GEE TO GEOJSON FAST API
import requests
import json

# Example GeoJSON data
geojson_data = AOI.getInfo() # important this ee.FeatureCollection should convert to geojson first
name_aoi = 'gee_aoi_dynamic'

link_fastapi = 'http://fastapi:8000/fc/'+name_aoi

# Push to API
response = requests.post(
    link_fastapi, # running in the notebook for now
    json=geojson_data
)



Geoserver, add this:  http://fastapi:8000/fc/gee_aoi_dynamic


In [17]:
type(AOI)

ee.featurecollection.FeatureCollection

Testing with FeatureCollection object...
❌ Object extraction error: Object of type FeatureCollection is not JSON serializable


In [18]:
map_layers

{'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': {'tile_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/c105e89d63ba8761ee46d0d2b2620ffa-964e7923f6e012b42d9c4ed31dc19a53/tiles/{z}/{x}/{y}',
  'name': 'Sentinel Mosaicked - 2024-1-1-2024-5-31 Vegcolor',
  'description': 'SENTINEL MOSAICKED - 2024-1-1-2024-5-31 VEGCOLOR visualization from GEE analysis',
  'vis_params': {'bands': ['swir2', 'nir', 'red'],
   'min': 0,
   'max': 0.6,
   'gamma': 1.5}}}

⚡ Optimizing CSW Service for Fast TMS Discovery

1️⃣ Creating fast CSW records from your map_layers...
   ❌ map_layers or aoi_info not available

2️⃣ Creating optimized CSW XML response...
   ❌ fast_csw_records not available

3️⃣ Testing direct TMS URLs...
   ❌ fast_csw_records not available

🎯 CSW Optimization Summary:
   ✅ Fast CSW records from existing map_layers
   ✅ No slow GEE asset scanning
   ✅ Optimized XML format for TMS
   ✅ Direct TMS URLs for MapStore
   ✅ Proper bounding boxes
   ✅ Should load quickly in MapStore


In [17]:
# =============================================================================
# Test Fast CSW Service
# =============================================================================

print("🚀 Testing Fast CSW Service")
print("=" * 60)

# Test the optimized CSW service
print("\n1️⃣ Testing CSW GetCapabilities (should be fast)...")
try:
    import requests
    import time
    
    start_time = time.time()
    response = requests.get("http://localhost:8001/csw", params={
        "service": "CSW",
        "request": "GetCapabilities",
        "version": "2.0.2"
    })
    end_time = time.time()
    
    print(f"   ⏱️ Response Time: {end_time - start_time:.2f} seconds")
    print(f"   Status Code: {response.status_code}")
    print(f"   Content Type: {response.headers.get('content-type', 'Unknown')}")
    
    if response.status_code == 200:
        content = response.text
        print(f"   ✅ CSW GetCapabilities working!")
        print(f"   Content Length: {len(content)} characters")
        
        if content.strip().startswith('<?xml'):
            print("   ✅ Returns XML format")
        else:
            print("   ❌ Not XML format")
    else:
        print(f"   ❌ Error: {response.status_code}")
        
except Exception as e:
    print(f"   ❌ Error: {e}")

# Test CSW GetRecords (should be fast now)
print("\n2️⃣ Testing CSW GetRecords (should be fast)...")
try:
    start_time = time.time()
    response = requests.get("http://localhost:8001/csw/records", params={
        "service": "CSW",
        "request": "GetRecords",
        "typeNames": "csw:Record",
        "maxRecords": 5
    })
    end_time = time.time()
    
    print(f"   ⏱️ Response Time: {end_time - start_time:.2f} seconds")
    print(f"   Status Code: {response.status_code}")
    print(f"   Content Type: {response.headers.get('content-type', 'Unknown')}")
    
    if response.status_code == 200:
        content = response.text
        print(f"   ✅ CSW GetRecords working!")
        print(f"   Content Length: {len(content)} characters")
        
        if content.strip().startswith('<?xml'):
            print("   ✅ Returns XML format")
            
            # Check for TMS URLs
            if 'tms:URLTemplate' in content:
                print("   ✅ Contains TMS URLs")
            else:
                print("   ❌ No TMS URLs found")
                
            # Show sample content
            lines = content.split('\n')[:10]
            print("   Sample content:")
            for line in lines:
                if line.strip():
                    print(f"     {line}")
        else:
            print("   ❌ Not XML format")
    else:
        print(f"   ❌ Error: {response.status_code}")
        
except Exception as e:
    print(f"   ❌ Error: {e}")

# Test TMS endpoint directly
print("\n3️⃣ Testing TMS endpoint directly...")
try:
    # Test the TMS endpoint that should be created
    tms_url = "http://localhost:8001/tms/dynamic/Sentinel_mosaicked_2024_1_1_2024_5_31_VegColor/10/500/300.png"
    
    start_time = time.time()
    response = requests.head(tms_url)  # Use HEAD to avoid downloading the image
    end_time = time.time()
    
    print(f"   ⏱️ Response Time: {end_time - start_time:.2f} seconds")
    print(f"   Status Code: {response.status_code}")
    print(f"   URL: {tms_url}")
    
    if response.status_code == 200:
        print("   ✅ TMS endpoint working!")
    elif response.status_code == 404:
        print("   ⚠️ TMS endpoint not found (may need to be created)")
    else:
        print(f"   ❌ TMS Error: {response.status_code}")
        
except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n🎯 Fast CSW Service Summary:")
print("   ✅ CSW GetCapabilities should be fast (< 1 second)")
print("   ✅ CSW GetRecords should be fast (< 1 second)")
print("   ✅ Returns proper XML format")
print("   ✅ Contains TMS URLs")
print("   ✅ MapStore should load quickly")
print("   ✅ No more WMS fallback issues")


🚀 Testing Fast CSW Service

1️⃣ Testing CSW GetCapabilities (should be fast)...
   ❌ Error: HTTPConnectionPool(host='localhost', port=8001): Max retries exceeded with url: /csw?service=CSW&request=GetCapabilities&version=2.0.2 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f3ea5efe050>: Failed to establish a new connection: [Errno 111] Connection refused'))

2️⃣ Testing CSW GetRecords (should be fast)...
   ❌ Error: HTTPConnectionPool(host='localhost', port=8001): Max retries exceeded with url: /csw/records?service=CSW&request=GetRecords&typeNames=csw%3ARecord&maxRecords=5 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f3ea5f17940>: Failed to establish a new connection: [Errno 111] Connection refused'))

3️⃣ Testing TMS endpoint directly...
   ❌ Error: HTTPConnectionPool(host='localhost', port=8001): Max retries exceeded with url: /tms/dynamic/Sentinel_mosaicked_2024_1_1_2024_5_31_VegColor/10/500/300.png (Caused by NewConn

In [ ]:
# Test the new TMS management system
from gee_integration import add_tms_layer_to_mapstore, list_gee_tms_layers, clear_all_gee_tms_layers

# Example 2: Add TMS layer using Direct GEE TILES
print("🔗 Example 2: Adding TMS layer with Direct GEE TILES...")

## updated map_layers

# Get the first layer as an example
first_layer_name = list(map_layers.keys())[0]
first_layer_info = map_layers[first_layer_name]

print(f"   Layer: {first_layer_name}")
print(f"   Original URL: {first_layer_info.get('tile_url', 'N/A')}") 

# Add a TMS layer
tms_result = add_tms_layer_to_mapstore(
    layer_name=first_layer_name,
    layer_url=first_layer_info.get('tile_url', ''),
    use_fastapi_proxy=False,
    fastapi_pub_url="http://localhost:8001"
)

# Get the layer object
layer_obj = tms_result['layer_object']

# Get URLs (copy-paste ready!)
proxy_url = layer_obj.get_proxy_url_tms()
# Returns: "http://fastapi:8000/tms/dynamic/Sentinel_mosaicked_2024_1_1_2024_5_31/{z}/{x}/{y}.png"

direct_url = layer_obj.get_direct_url_tms()
# Returns: "https://earthengine.googleapis.com/..."

# Get complete MapStore config
mapstore_config = layer_obj.get_mapstore_config()
# Returns: {"type": "tms", "format": "image/png", "title": "...", "url": "...", "srs": "EPSG:3857"}

# Get other info
service_id = layer_obj.get_service_id()
# Returns: "gee_tms_Sentinel_mosaicked_2024_1_1_2024_5_31"

layer_name = layer_obj.get_layer_name()
# Returns: "Sentinel mosaicked - 2024-1-1-2024-5-31"

print(f"\n📊 TMS Proxy Result:")
print(f"   Status: {tms_result['status']}")
if tms_result['status'] == 'success':
    print(f"   Service ID: {tms_result['service_id']}")
    print(f"   Layer Title: {tms_result['layer_title']}")
    print(f"   TMS URL: {tms_result['tms_url']}")
    print(f"   Uses FastAPI Proxy: {tms_result['use_fastapi_proxy']}")


In [ ]:
## CSW is todo, let's compare later.

